# Creazione schema mediato per ogni cluster

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
import json
import shutil

In [ ]:
def remove_and_make_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
        os.mkdir(path)
    else:
        os.mkdir(path)

In [ ]:
datasets_base_path = "..\\..\\Dataset\\ClusterParsed\\"
synonyms_path = ".\\synonyms\\"
schema_path = ".\\schema\\"

In [ ]:
remove_and_make_dir(schema_path)

In [ ]:
cluster_info = {}
for cluster_folder_name in os.listdir(datasets_base_path):
    cluster_path = os.path.join(datasets_base_path, cluster_folder_name)
    if len(os.listdir(cluster_path)) > 1:
        cluster_info[cluster_folder_name] = {}
        for filename in os.listdir(cluster_path):
            dataset_path = os.path.join(cluster_path, filename)
            cluster_info[cluster_folder_name][os.path.splitext(filename)[0]] = dataset_path

In [ ]:
cluster_synonyms = {}
for filename in os.listdir(synonyms_path):
    file_path = os.path.join(synonyms_path, filename)
    cluster = os.path.splitext(filename)[0]
    file = open(file_path)
    cluster_synonyms[cluster] = json.load(file)
    file.close()

In [ ]:
cluster_inverted_synonym_index = {}
for cluster, synonyms in cluster_synonyms.items():
    cluster_inverted_synonym_index[cluster] = {}
    for main_token, tokens in synonyms.items():
        for token in tokens:
            tmp = token.split("-")
            cluster_inverted_synonym_index[cluster][tmp[2]] = main_token

In [ ]:
cluster_schema = {cluster: synonyms.keys() for cluster, synonyms in cluster_synonyms.items()}

In [ ]:
cluster_group_match_column = {}
for cluster, groups in cluster_info.items():
    cluster_group_match_column[cluster] = {}
    for group, path in groups.items():
        cluster_group_match_column[cluster][path] = {}
        df = pd.read_csv(path)
        columns = list(df.columns)
        for column in columns:
            if column in cluster_inverted_synonym_index[cluster].keys():
                cluster_group_match_column[cluster][path][column] = cluster_inverted_synonym_index[cluster][column]

In [ ]:
for cluster, paths in cluster_group_match_column.items():
    df = pd.DataFrame(columns=cluster_schema[cluster])
    for path, column_rename in paths.items():
        df_tmp = pd.read_csv(path)
        df_tmp = df_tmp.rename(columns=column_rename)
        df_tmp = df_tmp[df_tmp.columns.intersection(column_rename.keys())]
        df = pd.concat([df, df_tmp], ignore_index=True)
    df.to_csv(schema_path+cluster+".csv", index=False)